<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/model_context_encoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GANime - Context Encoder
This notebook applies the context encoder paper.

## Imports and Parameters

In [ ]:
import os
import sys
import shutil
import glob
import random
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils

from google.colab import drive
drive.mount('/content/gdrive')

sys.path.append('/content/gdrive/MyDrive/repos/GANime/src')
from helper_functions import apply_mask, apply_padding, apply_comp, apply_scale, load_checkpoint, checkpoint
from data_loaders import create_dataloaders
from networks_context_encoders import Generator, Discriminator, weights_init

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# network parameters
BATCH_SIZE = 15
DATASET_SIZE = 100000
N_BATCHES = DATASET_SIZE // BATCH_SIZE
N_GPU = 1
N_WORKERS = 1
N_EPOCHS = 100
LEARNING_RATE = 0.0002
DROPOUT_RATE = 0.2

# image
IMG_HEIGHT = 256
IMG_WIDTH = 456
SINGLE_SIDE = 57

# tensorboard
TRAIN_REFERENCE_INDEX = 200
VAL_REFERENCE_INDEX = 100
TEST_REFERENCE_INDEX = 20

# cost weights
WEIGHT_DECAY = 0.05

# directories
ZIP_PATH_TRAIN = '/content/gdrive/My Drive/repos/GANime/data_out/train.zip'
IMG_DIR_TRAIN = '/content/frames/train/'
ZIP_PATH_VAL = '/content/gdrive/My Drive/repos/GANime/data_out/validate.zip'
IMG_DIR_VAL = '/content/frames/validate/'
ZIP_PATH_TEST = '/content/gdrive/My Drive/repos/GANime/data_out/test.zip'
IMG_DIR_TEST = '/content/frames/test/'
LOG_DIR = '/content/gdrive/My Drive/repos/GANime/data_out/logs/model_gans/'
PREV_CHECKPOINT = '/content/gdrive/My Drive/repos/GANime/data_out/logs/model_gans/checkpoint.pt' # set to none to not load and create a new log folder
# PREV_CHECKPOINT = None # set to none to not load and create a new log folder